In [2]:
!nvidia-smi -L
from google.colab import drive
drive.mount('/content/drive')


GPU 0: Tesla K80 (UUID: GPU-4f41db11-4928-03c8-8a30-3f37ffbbf893)
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Imports

In [3]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
from __future__ import unicode_literals

import glob
import tensorflow as tf
import numpy as np
import random
import os
import sys

import time

import numpy as np

from math import atan, sqrt
from tensorflow import keras

!pip show tensorflow

Name: tensorflow
Version: 1.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: keras-preprocessing, tensorboard, wrapt, six, absl-py, termcolor, grpcio, protobuf, keras-applications, tensorflow-estimator, google-pasta, wheel, astor, opt-einsum, gast, numpy
Required-by: stable-baselines, magenta, fancyimpute


# Set Constants

In [0]:
import os
PROJECT_ROOT = "/content/drive/My Drive/rob535"
TEST_FILES = "/content/drive/My Drive/rob535/test"
TRAIN_FILES = "/content/drive/My Drive/rob535/trainval"

STAGE_1 = os.path.join(PROJECT_ROOT, "stage1")
os.chdir(PROJECT_ROOT)

# Training


## Generate Trainng Data

In [0]:
def rot(n):
    n = np.asarray(n).flatten()
    assert(n.size == 3)

    theta = np.linalg.norm(n)
    if theta:
        n /= theta
        K = np.array([[0, -n[2], n[1]], [n[2], 0, -n[0]], [-n[1], n[0], 0]])

        return np.identity(3) + np.sin(theta) * K + (1 - np.cos(theta)) * K @ K
    else:
        return np.identity(3)


def get_bbox(p0, p1):
    v = np.array([
        [p0[0], p0[0], p0[0], p0[0], p1[0], p1[0], p1[0], p1[0]],
        [p0[1], p0[1], p1[1], p1[1], p0[1], p0[1], p1[1], p1[1]],
        [p0[2], p1[2], p0[2], p1[2], p0[2], p1[2], p0[2], p1[2]]
    ])
    e = np.array([
        [2, 3, 0, 0, 3, 3, 0, 1, 2, 3, 4, 4, 7, 7],
        [7, 6, 1, 2, 1, 2, 4, 5, 6, 7, 5, 6, 5, 6]
    ], dtype=np.uint8)

    return v, e


def check_coords(xmin, xmax, ymin, ymax):
    height = 1052  # Image height
    width = 1914  # Image width

    if xmin < 0:
        xmin = 0
    if xmax > width:
        xmax = width
    if ymin < 0:
        ymin = 0
    if ymax > height:
        ymax = height

    return xmin, xmax, ymin, ymax


def create_nparray(bbox_paths, istrain):
    X = np.zeros((10, 0))
    Y = np.zeros((2, 0))
    file_map = np.array([])

    num_in = 0
    start = time.time()
    for i in range(0, len(bbox_paths)):
        if not i % 100:
            print("%d/%d\t%f s" % (i, len(bbox_paths), time.time()-start))
        proj = np.fromfile(bbox_paths[i].replace("_bbox.bin", "_proj.bin"), dtype=np.float32)
        proj.resize([3, 4])
        bbox = np.fromfile(bbox_paths[i], dtype=np.float32)
        bbox = bbox.reshape([-1, 11])
        xyz = np.fromfile(bbox_paths[i].replace('_bbox.bin', '_cloud.bin'), dtype=np.float32)
        xyz = xyz.reshape([3, -1])
        uv = proj @ np.vstack([xyz, np.ones_like(xyz[0, :])])
        uv = uv / uv[2, :]
        dist = np.linalg.norm(xyz, axis=0)

        for k, b in enumerate(bbox):
            file_map = np.append(file_map, bbox_paths[i])
            R = rot(b[0:3])
            t = b[3:6]
            sz = b[6:9]

            vert_3D, edges = get_bbox(-sz / 2, sz / 2)
            vert_3D = R @ vert_3D + t[:, np.newaxis]

            vert_2D = proj @ np.vstack([vert_3D, np.ones(vert_3D.shape[1])])
            vert_2D = vert_2D / vert_2D[2, :]

            xmin = min(vert_2D[0, :])
            xmax = max(vert_2D[0, :])
            ymin = min(vert_2D[1, :])
            ymax = max(vert_2D[1, :])

            xmin, xmax, ymin, ymax = check_coords(xmin, xmax, ymin, ymax)
            label = int(b[9]) #use 0 index

            x_in_box = []
            y_in_box = []
            dist_box = []
            for j in range(0, uv.shape[1]):
                x = uv[0,j]
                y = uv[1,j]
                if xmin < x < xmax and ymin < y < ymax:
                    x_in_box.append(x)
                    y_in_box.append(y)
                    dist_box.append(dist[j])

            if len(dist_box) < 10:
                #ignore if too few points (too far away)
                continue

            X = np.append(X, np.zeros((10, 1)), axis=1)
            X[0, num_in] = xmin
            X[1, num_in] = xmax
            X[2, num_in] = ymin
            X[3, num_in] = ymax
            X[4, num_in] = min(dist_box)
            X[5, num_in] = max(dist_box)
            X[6, num_in] = np.mean(dist_box)
            X[7, num_in] = np.median(dist_box)
            X[8, num_in] = np.std(dist_box)
            X[9, num_in] = label

            #outputs
            x_cent = sum(vert_3D[0,:])/8
            y_cent = sum(vert_3D[1,:])/8
            z_cent = sum(vert_3D[2,:])/8
            r = sqrt(x_cent*x_cent + y_cent*y_cent + z_cent*z_cent)
            theta = atan(x_cent/z_cent)

            Y = np.append(Y, np.zeros((2,1)), axis=1)
            Y[0, num_in] = r
            Y[1, num_in] = theta

            num_in = num_in + 1

    if istrain:
        x_file = "stage2/X_train"
        y_file = "stage2/Y_train"
    else:
        x_file = "stage2/X_test"
        y_file = "stage2/Y_test"

    np.save(x_file, X, allow_pickle=True)
    np.save(y_file, Y, allow_pickle=True)
    np.save("stage2/filename_map", file_map, allow_pickle=True)



In [6]:
bbox_path = glob.glob(os.path.join(TRAIN_FILES, "*/*bbox.bin"))
random.shuffle(bbox_path)
num_test = int(0.2*len(bbox_path))
test_paths = bbox_path[0:num_test]
train_paths = bbox_path[num_test:]
print(num_test)
create_nparray(test_paths, False)
create_nparray(train_paths, True)


1112
0/1112	0.000004 s
100/1112	106.406191 s
200/1112	210.810164 s
300/1112	324.493070 s
400/1112	432.981932 s
500/1112	541.936831 s
600/1112	643.810665 s
700/1112	751.331069 s
800/1112	866.051366 s
900/1112	971.298912 s
1000/1112	1078.905690 s
1100/1112	1184.496230 s
0/4449	0.000008 s
100/4449	111.360157 s
200/4449	213.313435 s
300/4449	318.752999 s
400/4449	419.199434 s
500/4449	518.213651 s
600/4449	619.644658 s
700/4449	726.982475 s
800/4449	830.749574 s
900/4449	934.122754 s
1000/4449	1036.233771 s
1100/4449	1138.131327 s
1200/4449	1242.001626 s
1300/4449	1344.501839 s
1400/4449	1448.983827 s
1500/4449	1551.450983 s
1600/4449	1653.505356 s
1700/4449	1753.620491 s
1800/4449	1853.452803 s
1900/4449	1953.225331 s
2000/4449	2069.028049 s
2100/4449	2191.586748 s
2200/4449	2310.214009 s
2300/4449	2413.913532 s
2400/4449	2512.377194 s
2500/4449	2617.391236 s
2600/4449	2723.200260 s
2700/4449	2827.715573 s
2800/4449	2929.721024 s
2900/4449	3035.937360 s
3000/4449	3139.080041 s
3100/4449	3

## Train Data

In [7]:
def avg_dist_error(y_true, y_pred):
    error = tf.math.abs(tf.math.subtract(y_true[:,0], y_pred[:,0]))
    return tf.math.reduce_mean(error)


def avg_angle_error(y_true, y_pred):
    error = tf.math.abs(tf.math.subtract(y_true[:,1], y_pred[:,1]))
    return tf.math.reduce_mean(error)

def to_class_label_4(num):
    if num == 0 or num == 15 or num == 16 or num == 17 or num == 22:
        return 0
    elif 1 <= num <= 8:
        return 1
    elif num == 9 or num == 10 or num == 14:
        return 3
    else:
        return 2

THRESHOLD = 55

X_train_orig = np.transpose(np.load("stage2/X_train.npy"))
Y_train_orig = np.transpose(np.load("stage2/Y_train.npy"))
X_train = X_train_orig
Y_train = Y_train_orig
X_test = np.transpose(np.load("stage2/X_test.npy"))
Y_test = np.transpose(np.load("stage2/Y_test.npy"))

#fix radians to degrees
Y_train[:, 1] = Y_train[:, 1]*180/np.pi
Y_test[:, 1] = Y_test[:, 1]*180/np.pi

del_list = []
#remove anything > THRESHOLD
for i in range(0, X_train.shape[0]):
    if Y_train[i, 0] > THRESHOLD:
        del_list.append(i)

#fix labels to 0-3
for i in range(0, X_train.shape[0]):
    X_train[i, 9] = to_class_label_4(X_train[i, 9])

for i in range(0, X_test.shape[0]):
    X_test[i, 9] = to_class_label_4(X_test[i, 9])

X_train = np.delete(X_train, del_list, axis=0)
Y_train = np.delete(Y_train, del_list, axis=0)

model = keras.Sequential([
    keras.layers.Dense(16, input_dim=10, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(2)
])

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=[avg_dist_error, avg_angle_error])

model.fit(X_train, Y_train, epochs=200)

Y_predict = model.predict(X_test)

np.savetxt("stage2/Y_predict.txt", Y_predict, fmt="%f")
np.savetxt("stage2/Y_truth.txt", Y_test, fmt="%f")

dist_errs = []
theta_errs = []
for i in range(0, Y_predict.shape[0]):
    if Y_test[i, 0] <= 50:
        dist_errs.append(Y_test[i, 0] - Y_predict[i, 0])
        theta_errs.append(Y_test[i, 1] - Y_predict[i, 1])

print(np.mean(np.abs(dist_errs)))
print(np.mean(np.abs(theta_errs)))

model.save("stage2/model.h5")

with open('stage2/model_architecture.json', 'w') as f:
    f.write(model.to_json())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 4082 samples
Epoch 1/200
4082/4082 [==============================] - 2s 396us/sample - loss: 418.8011 - avg_dist_error: 12.8212 - avg_angle_error: 9.9496
Epoch 2/200
4082/4082 [==============================] - 1s 135us/sample - loss: 54.1372 - avg_dist_error: 5.4621 - avg_angle_error: 4.3371
Epoch 3/200
4082/4082 [==============================] - 1s 140us/sample - loss: 42.7061 - avg_dist_error: 4.4572 - avg_angle_error: 3.8725
Epoch 4/200
4082/4082 [==============================] - 1s 137us/sample - loss: 43.3317 - avg_dist_error: 4.2889 - avg_angle_error: 4.2234
Epoch 5/200
4082/4082 [==============================] - 1s 143us/sample - loss: 40.3299 - avg_dist_error: 4.0581 - avg_angle_error: 4.0295
Epoch 6/200
4082/4082 [==============================] - 1s 148us/sample - loss: 39.3957 - avg_dist_error: 4.1255 - avg_angle_error: 3.8551
Epoch 7/200
4082/4082 [==============================] 

# Evaluation

## Preproccess Stage 1 Data

In [11]:
import pickle

def to_class_label_4(num):
    if num == 0 or num == 15 or num == 16 or num == 17 or num == 22:
        return 0
    elif 1 <= num <= 8:
        return 1
    elif num == 9 or num == 10 or num == 14:
        return 3
    else:
        return 2


def get_local_path(gdrive_path):
    gdrive_path = gdrive_path.split("/")
    local_path = os.path.join(TEST_FILES,
                              gdrive_path[-2], gdrive_path[-1])
    local_path = local_path.replace("_compressed.jpg", ".jpg")
    return local_path


def load_data_dwight(file_path): #TODO
    data = np.load(file_path, allow_pickle=True)
    X_test = np.array(np.zeros((0,10)))
    filename_map = np.array([])
    num_in = 0
    for i in range(0, data.shape[0]):
        local_path = get_local_path(data[i][0])
        bbox_list = data[i][1]

        proj = np.fromfile(local_path.replace("_image.jpg", "_proj.bin"), dtype=np.float32)
        proj.resize([3, 4])
        xyz = np.fromfile(local_path.replace('_image.jpg', '_cloud.bin'), dtype=np.float32)
        xyz = xyz.reshape([3, -1])
        uv = proj @ np.vstack([xyz, np.ones_like(xyz[0, :])])
        uv = uv / uv[2, :]
        dist = np.linalg.norm(xyz, axis=0)

        for bbox in bbox_list:
            if bbox["name"] == "car":
                label = 1
            elif bbox["name"] == "truck" or bbox["name"] == "bus":
                label = 2
            elif bbox["name"] == "motorbike" or bbox["name"] == "bicycle":
                label = 3
            else:
                continue

            xmin = bbox["box_points"][0]
            ymin = bbox["box_points"][1]
            xmax = bbox["box_points"][2]
            ymax = bbox["box_points"][3]

            dist_box = []
            for j in range(0, uv.shape[1]):
                x = uv[0,j]
                y = uv[1,j]
                if xmin < x < xmax and ymin < y < ymax:
                    dist_box.append(dist[j])

            if len(dist_box) < 10:
                #ignore if too few points (too far away)
                continue

            filename_map = np.append(filename_map, local_path)
            X_test = np.append(X_test, np.zeros((1, 10)), axis=0)

            X_test[num_in, 0] = xmin
            X_test[num_in, 1] = xmax
            X_test[num_in, 2] = ymin
            X_test[num_in, 3] = ymax
            X_test[num_in, 4] = min(dist_box)
            X_test[num_in, 5] = max(dist_box)
            X_test[num_in, 6] = np.mean(dist_box)
            X_test[num_in, 7] = np.median(dist_box)
            X_test[num_in, 8] = np.std(dist_box)
            X_test[num_in, 9] = label

            num_in = num_in + 1

    return X_test, filename_map


def load_data_chris(detections_file_path):
    detections = pickle.load( open(detections_file_path, "rb"))
    width = 1914
    height = 1052
    X_test = np.array(np.zeros((0,10)))
    filename_map = np.array([])

    num_in = 0
    start = time.time()
    for k, one_image in enumerate(detections):
        if not k % 100:
            print("%d/%d\t%f s" % (k, len(detections), time.time()-start))

        local_path = get_local_path(one_image["path"])
        bbox = one_image["bbox"]

        proj = np.fromfile(local_path.replace("_image.jpg", "_proj.bin"), dtype=np.float32)
        proj.resize([3, 4])
        xyz = np.fromfile(local_path.replace('_image.jpg', '_cloud.bin'), dtype=np.float32)
        xyz = xyz.reshape([3, -1])
        uv = proj @ np.vstack([xyz, np.ones_like(xyz[0, :])])
        uv = uv / uv[2, :]
        dist = np.linalg.norm(xyz, axis=0)

        for i in range(0, len(bbox)):
            label = one_image["label"][i]-1

            #hopefully these are right
            xmin = min(bbox[i][1], bbox[i][3]) * width
            xmax = max(bbox[i][1], bbox[i][3]) * width
            ymin = min(bbox[i][0], bbox[i][2]) * height
            ymax = max(bbox[i][0], bbox[i][2]) * height

            dist_box = []
            for j in range(0, uv.shape[1]):
                x = uv[0, j]
                y = uv[1, j]
                if xmin < x < xmax and ymin < y < ymax:
                    dist_box.append(dist[j])

            if len(dist_box) < 10:
                # ignore if too few points (too far away)
                continue

            filename_map = np.append(filename_map, local_path)
            X_test = np.append(X_test, np.zeros((1, 10)), axis=0)

            X_test[num_in, 0] = xmin
            X_test[num_in, 1] = xmax
            X_test[num_in, 2] = ymin
            X_test[num_in, 3] = ymax
            X_test[num_in, 4] = min(dist_box)
            X_test[num_in, 5] = max(dist_box)
            X_test[num_in, 6] = np.mean(dist_box)
            X_test[num_in, 7] = np.median(dist_box)
            X_test[num_in, 8] = np.std(dist_box)
            X_test[num_in, 9] = label

            num_in = num_in + 1

    return X_test, filename_map


DWIGHT = False

if DWIGHT:
    X_test, filename_map = load_data_dwight("stage2/detections_backup.npy")
    np.save("stage2/filename_map_eval_dwight", filename_map, allow_pickle=True)
    np.save("stage2/X_test_eval_dwight", X_test, allow_pickle=True)
else:
    X_test, filename_map = load_data_chris("stage2/calculated_bbox-test.pkl")
    np.save("stage2/filename_map_eval", filename_map, allow_pickle=True)
    np.save("stage2/X_test_eval", X_test, allow_pickle=True)


0/1799	0.000016 s
100/1799	23.973445 s
200/1799	73.710057 s
300/1799	153.052218 s
400/1799	236.721953 s
500/1799	331.473822 s
600/1799	404.006353 s
700/1799	480.214660 s
800/1799	561.839817 s
900/1799	641.074482 s
1000/1799	720.091581 s
1100/1799	793.591097 s
1200/1799	867.258661 s
1300/1799	943.618031 s
1400/1799	1013.528819 s
1500/1799	1086.420607 s
1600/1799	1165.711632 s
1700/1799	1240.276941 s


## Evaluate and generate CSV Files

In [0]:
import csv

with open('stage2/model_architecture.json', 'r') as f:
    model = keras.models.model_from_json(f.read())

model.load_weights("stage2/model.h5")

X_test = np.load("stage2/X_test_eval.npy")
filename_map = np.load("stage2/filename_map_eval.npy")

Y_predict = model.predict(X_test)
np.savetxt("stage2/Y_predict_eval.txt", Y_predict, fmt="%f")

#create task 1 csv
data_dir = TEST_FILES
image_files_full = glob.glob(os.path.join(data_dir, "*/*_image.jpg"))

image_files = []
pred_dict = {} #maps filename -> closest detection
for i, image_file in enumerate(image_files_full):
    path_split = image_file.split(os.sep)
    image_files.append(os.path.join(path_split[-2], path_split[-1]).replace(os.sep, "/"))
    pred_dict[image_files[i]] = []

for i in range(0, Y_predict.shape[0]):
    path_split = filename_map[i].split(os.sep)
    file = os.path.join(path_split[-2], path_split[-1]).replace(os.sep, "/")

    if len(pred_dict[file]) == 0 or pred_dict[file][0]["dist"] > Y_predict[i, 0]:
        tmp = {"dist": Y_predict[i, 0], "angle": Y_predict[i, 1], "label": int(X_test[i, 9]),
               "bbox": [X_test[i,0], X_test[i,1], X_test[i,2], X_test[i,3]]}
        try:
            pred_dict[file][0] = tmp
        except IndexError:
            pred_dict[file].append(tmp)

pred_labels = []
for image_file in image_files:
    if len(pred_dict[image_file]) == 0 or pred_dict[image_file][0]["dist"] > 50:
        pred_labels.append(0)
    else:
        pred_labels.append(pred_dict[image_file][0]["label"])

with open("stage2/results-trained-resnet101-task1.csv", "w+") as results_fd:
    results_writer = csv.writer(results_fd, delimiter=",", lineterminator="\n")
    results_writer.writerow(["guid/image", "label"])
    for i, image_file in enumerate(image_files):
        results_writer.writerow([image_file.replace("_image.jpg", ""), pred_labels[i]])

#Create task2 csv
with open("stage2/template.csv") as template_fd:
    template_reader = csv.reader(template_fd, delimiter=",")
    image_files_task2 = []
    for i, row in enumerate(template_reader):
        if i % 2 == 0:
            continue
        else:
            impath = row[0].replace("/r", "_image.jpg")
            image_files_task2.append(impath)

with open("stage2/results-trained-resnet101-task2.csv", "w+") as results_fd:
    results_writer = csv.writer(results_fd, delimiter=",", lineterminator="\n")
    results_writer.writerow(["guid/image/axis", "value"])
    for i, image_file in enumerate(image_files_task2):
        result_val = pred_dict[image_file]
        if len(result_val) != 0:
            results_writer.writerow([image_file.replace("_image.jpg", "") + "/r",
                                     result_val[0]["dist"]])
            results_writer.writerow([image_file.replace("_image.jpg", "") + "/theta",
                                     result_val[0]["angle"]])
        else:
            results_writer.writerow([image_file.replace("_image.jpg", "") + "/r",
                                     25])
            results_writer.writerow([image_file.replace("_image.jpg", "") + "/theta",
                                     0])
